In [1]:
from pathlib import Path

from pymongo.results import InsertOneResult

from twitter_trends import ArgsTwitterTrends, ParamsTwitterTrends, WOEIDCountry
from scraping_kit.db.models.raw import RawData
from scraping_kit.db.models.trends import Trends
from scraping_kit.db.db_twitter import DBTwitter
from scraping_kit import load_db_and_bots, BotScraper

path_data = Path("data")
db_tw, bots = load_db_and_bots(path_data, "scrape_tw")

Collection Names: ['trends', 'raw']
Bots: bots=[BotScraper(acc_name='Vera')]


In [ ]:
def process_trends_raw_data(db_tw: DBTwitter, insert_one_result_raw: InsertOneResult) -> InsertOneResult:
    raw_data = db_tw.coll.raw.find_one(insert_one_result_raw.inserted_id)
    raw_data = RawData(**raw_data)
    trends = Trends.from_raw(raw_data)
    insert_one_result_trend = db_tw.coll.trends.insert_one(trends.model_dump())     # TODO: Check its ok.
    return insert_one_result_trend

def requests_trends_and_save(db_tw: DBTwitter, req_args: ArgsTwitterTrends, bot: BotScraper) -> InsertOneResult:
    response, creation_date = bot.get_response(req_args)
    insert_one_result_raw = db_tw.add_raw_data(req_args, response, creation_date)
    return insert_one_result_raw

def requests_and_process(db_tw: DBTwitter, req_args: ArgsTwitterTrends, bot: BotScraper) -> InsertOneResult:
    insert_one_result_raw = requests_trends_and_save(db_tw, req_args, bot)
    insert_one_result_trend = process_trends_raw_data(db_tw, insert_one_result_raw)
    db_tw.coll.change_is_processed(insert_one_result_raw)
    return insert_one_result_trend

In [ ]:
req_args = ArgsTwitterTrends(data=ParamsTwitterTrends(woeid=WOEIDCountry.united_states))    # You can change woeid if you need.
idx_bot_choiced, bot_choiced = bots.random_bot()

insert_one_result_trend = requests_and_process(db_tw, req_args, bot_choiced)
insert_one_result_trend